## Overview of `uresnet.cfg` Configuration File

The `uresnet.cfg` configuration file defines how the UResNet model will be trained using **SPINE** for a semantic segmentation task. Below is a breakdown of the key components.

### 1. Base Configuration
This section defines general settings for the training process:

- **`world_size: 1`**: Specifies the number of GPUs to use (1 GPU in this case).
- **`iterations: 25000`**: The number of training iterations (~25 epochs).
- **`seed: 0`**: A random seed for reproducibility.
- **`unwrap: false`**: Do not break down input/output images within the batch (this is not necessary for training).
- **`log_dir: logs/uresnet`**: The directory where training logs will be saved.
- **`log_step: 1`**: Log training information after every iteration.
- **`overwrite_log: true`**: Overwrite existing logs if they already exist.

Within the `train` block:
- **`weight_prefix: weights/uresnet/snapshot`**: Specifies where to save model weights.
- **`save_step: 1000`**: Save the model weights every 1,000 iterations.
- **`optimizer` block**:
  - **`name: Adam`**: Use the Adam optimizer for training.
  - **`lr: 0.001`**: Learning rate for the optimizer.

### 2. IO (Input/Output) Configuration
This section controls how the data will be loaded for training:

- **`loader` block**:
  - **`batch_size: 128`**: The size of each training batch.
  - **`shuffle: false`**: Data shuffling is disabled (randomization is done via a sampler).
  - **`num_workers: 4`**: Number of CPU cores used for data loading.
  - **`collate_fn: all`**: Specifies how to combine data samples into batches.
  - **`sampler: random_sequence`**: Load data using a random sequence.
  
- **`dataset` block**:
  - **`name: larcv`**: Specifies the dataset format.
  - **`file_keys: /sdf/data/neutrino/generic/mpvmpr_2020_01_v04/train.root`**: Path to the training dataset.
  - **`schema` block**:
    - **`data`**:
      - **`parser: sparse3d`**: Specifies that the data format is sparse3D.
      - **`sparse_event: sparse3d_pcluster`**: Label for sparse 3D cluster data.
    - **`seg_label`**:
      - **`parser: sparse3d`**: Specifies the format for segmentation labels.
      - **`sparse_event: sparse3d_pcluster_semantics`**: Label for sparse 3D segmentation.

### 3. Model Configuration
This section defines the UResNet model:

- **`name: uresnet`**: Specifies the UResNet model to be used.
- **`weight_path: null`**: No pretrained weights are used for this model.
  
- **`network_input` block**:
  - **`data: data`**: Specifies the input data for the network (defined in the `IO` block).
  
- **`loss_input` block**:
  - **`seg_label: seg_label`**: Specifies the segmentation labels used in the loss function.

### 4. Modules
This section describes the architecture of the UResNet model and its loss function:

- **`uresnet` block**:
  - **`num_input: 1`**: Number of input channels (1 for grayscale images).
  - **`num_classes: 5`**: The number of output classes.
  - **`filters: 32`**: Number of filters in the first convolutional layer.
  - **`depth: 5`**: The depth of the UResNet architecture (5 layers).
  - **`reps: 2`**: Each layer is repeated twice.
  - **`allow_bias: false`**: No bias term is used in the convolutional layers.
  - **`activation` block**:
    - **`name: lrelu`**: Leaky ReLU activation function.
    - **`negative_slope: 0.33`**: Slope for the Leaky ReLU function.
  - **`norm_layer` block**:
    - **`name: batch_norm`**: Use batch normalization.
    - **`eps: 0.0001`**: Epsilon value to prevent division by zero.
    - **`momentum: 0.01`**: Momentum for the moving average in batch normalization.
  
- **`uresnet_loss` block**:
  - **`balance_loss: false`**: The loss function is not balanced across different classes.

### Purpose
The `uresnet.cfg` file defines the training process for a **UResNet model**, which is used for **3D sparse image segmentation**. The model is configured to run for 25,000 iterations, with model checkpoints saved every 1,000 iterations. It also logs the training progress after each iteration and uses the Adam optimizer to adjust the model weights.


## Overview of `uresnet_val.cfg` Configuration File

The `uresnet_val.cfg` file defines how the UResNet model will be validated using **SPINE**. It shares many similarities with the training configuration but is tailored for the validation process.

### 1. Base Configuration
This section defines general settings for the validation process:

- **`world_size: 1`**: Specifies the number of GPUs to use (1 GPU in this case).
- **`iterations: 100`**: The number of iterations for validation (~0.5 epochs).
- **`seed: 0`**: A random seed for reproducibility.
- **`unwrap: false`**: This setting should remain `false` during validation to avoid unnecessary operations.
- **`log_dir: logs/uresnet`**: Specifies the directory where logs will be saved.
- **`log_step: 1`**: Log validation information after every iteration.
- **`overwrite_log: true`**: If set to `true`, existing logs will be overwritten.

### 2. IO (Input/Output) Configuration
This section controls how the data is loaded for validation:

- **`loader` block**:
  - **`batch_size: 128`**: Specifies the size of each validation batch.
  - **`shuffle: false`**: Shuffling is turned off for validation to ensure deterministic evaluation.
  - **`num_workers: 4`**: Number of CPU cores used for data loading.
  - **`collate_fn: all`**: Specifies how data samples are combined into batches.
  
- **`dataset` block**:
  - **`name: larcv`**: Specifies the format of the dataset.
  - **`file_keys: /sdf/data/neutrino/generic/mpvmpr_2020_01_v04/test.root`**: Path to the validation dataset.
  - **`schema` block**:
    - **`data`**:
      - **`parser: sparse3d`**: Specifies how to parse the data (sparse3D format).
      - **`sparse_event: sparse3d_pcluster`**: Label for sparse 3D cluster data.
    - **`seg_label`**:
      - **`parser: sparse3d`**: Specifies how to parse segmentation labels.
      - **`sparse_event: sparse3d_pcluster_semantics`**: Label for sparse 3D segmentation labels.

### 3. Model Configuration
This section defines the model and how the validation will use it:

- **`name: uresnet`**: Specifies the UResNet model.
- **`weight_path: weights/uresnet/snapshot*.ckpt`**: Specifies the path to the saved model weights that will be used for validation.

- **`network_input` block**:
  - **`data: data`**: Defines the input data for the network.
  
- **`loss_input` block**:
  - **`seg_label: seg_label`**: Specifies the segmentation labels for validation.

### 4. Modules
This section describes the UResNet model architecture for validation:

- **`uresnet` block**:
  - **`num_input: 1`**: Number of input channels (1 for grayscale images).
  - **`num_classes: 5`**: Number of output classes.
  - **`filters: 32`**: Number of filters in the first convolutional layer.
  - **`depth: 5`**: The depth of the UResNet architecture (5 layers).
  - **`reps: 2`**: Each layer is repeated twice.
  - **`allow_bias: false`**: No bias term is used in the convolutional layers.
  - **`activation` block**:
    - **`name: lrelu`**: Leaky ReLU activation function.
    - **`negative_slope: 0.33`**: Slope for the Leaky ReLU function.
  - **`norm_layer` block**:
    - **`name: batch_norm`**: Applies batch normalization.
    - **`eps: 0.0001`**: Epsilon for numerical stability in batch normalization.
    - **`momentum: 0.01`**: Momentum for batch normalization.

- **`uresnet_loss` block**:
  - **`balance_loss: false`**: The loss function is not balanced across different classes.

### Purpose
The `uresnet_val.cfg` configuration file is specifically designed for validating the performance of a **UResNet model** on a test dataset. It loads previously saved model weights, runs the model for 100 iterations on the test data, and logs the validation results. This configuration ensures that the model's performance is evaluated in a controlled, deterministic environment.


## Overview of `grappa_shower.cfg` Configuration File

The `grappa_shower.cfg` file defines the training configuration for a **GRAPPA**-based model using **SPINE**. This file configures how the model will process 3D cluster and particle data for training.

### 1. Base Configuration
This section defines general training settings:

- **`world_size: 1`**: Specifies the number of GPUs to use (in this case, 1 GPU).
- **`iterations: 25000`**: The total number of training iterations (~25 epochs).
- **`seed: 0`**: Random seed for reproducibility.
- **`unwrap: false`**: Disables breaking down input/output images in the batch (saves time during training).
- **`log_dir: logs`**: Directory where training logs will be stored.
- **`log_step: 1`**: Logging frequency, logs will be written every iteration.
- **`overwrite_log: true`**: Allows overwriting existing logs.
  
- **`train` block**:
  - **`weight_prefix: weights/grappa_shower/snapshot-`**: Path where model weights will be saved.
  - **`save_step: 1000`**: Specifies how frequently to save model weights (every epoch in this case).
  - **`optimizer` block**:
    - **`name: Adam`**: Specifies the use of the Adam optimizer.
    - **`lr: 0.001`**: The learning rate for the optimizer.

### 2. IO (Input/Output) Configuration
This section defines how data is loaded during training:

- **`loader` block**:
  - **`batch_size: 128`**: The batch size for training.
  - **`shuffle: false`**: Data shuffling is disabled (randomization is handled by a custom sampler).
  - **`num_workers: 8`**: Number of CPU cores used for data loading.
  - **`collate_fn: all`**: Specifies how to combine individual data samples into batches.
  - **`sampler: random_sequence`**: Uses a random sequence sampler for loading data.

- **`dataset` block**:
  - **`name: larcv`**: Specifies the format of the dataset.
  - **`file_keys: /sdf/data/neutrino/generic/mpvmpr_2020_01_v04/train.root`**: Path to the training dataset.
  
  - **`schema` block**:
    - **`data` block**:
      - **`parser: cluster3d`**: Specifies how to parse the 3D cluster data.
      - **`cluster_event: cluster3d_pcluster`**: Event label for 3D cluster data.
      - **`particle_event: particle_corrected`**: Event label for corrected particle data.
      - **`sparse_semantics_event: sparse3d_pcluster_semantics`**: Event label for semantic sparse 3D cluster data.
      - **`add_particle_info: true`**: Enables adding additional particle information.
      - **`clean_data: true`**: Specifies whether to clean the data before processing.
      
    - **`coord_label` block**:
      - **`parser: particle_coords`**: Parser for particle coordinates.
      - **`particle_event: particle_corrected`**: Event label for corrected particle data.
      - **`cluster_event: cluster3d_pcluster`**: Event label for the 3D cluster data.

### 3. Model Configuration
This section defines the **GRAPPA** model and its components:

- **`name: grappa`**: Specifies the GRAPPA model.
- **`weight_path: null`**: No pretrained weights are used for the training.

- **`network_input` block**:
  - **`data: data`**: Defines the input data for the network.
  - **`coord_label: coord_label`**: Specifies the coordinate labels as input.
  
- **`loss_input` block**:
  - **`clust_label: data`**: Specifies the cluster label to be used for calculating loss.

### 4. Modules
This section defines the specific components of the GRAPPA model and its architecture:

- **`grappa` block**:
  - **`nodes` block**:
    - **`source: cluster`**: Specifies that the source is the cluster data.
    - **`shapes: [shower, michel, delta]`**: Specifies the types of shapes to identify: shower, Michel electron, and delta.
    - **`min_size: -1`**: Minimum size filter for the clusters (set to -1, no filtering).
    - **`make_groups: false`**: Groups are not created automatically.
    - **`grouping_method: score`**: Clusters are grouped based on a scoring mechanism.
    
  - **`graph` block**:
    - **`name: complete`**: Defines a complete graph for connecting the nodes.
    - **`max_length`**: Maximum distance between nodes for creating edges.

  - **`node_encoder` block**:
    - **`name: geo`**: Specifies the geometry-based node encoder.
    - Various options like `add_value`, `add_shape`, `add_points`, etc. allow adding different geometric features to the node representation.

  - **`edge_encoder` block**:
    - **`name: geo`**: Specifies a geometry-based edge encoder.

  - **`gnn_model` block**:
    - **`name: meta`**: Specifies the use of a meta-learning GNN model.
    - **`node_feats: 33`**: Number of features for each node.
    - **`edge_feats: 19`**: Number of features for each edge.
    - **`node_pred: 2`**: Number of output predictions for each node.
    - **`edge_pred: 2`**: Number of output predictions for each edge.
    - **`edge_layer` block**:
      - Specifies a multilayer perceptron (MLP) with 3 layers for predicting edges.
    - **`node_layer` block**:
      - Specifies an MLP for predicting node states with options for normalization, aggregation, and activation functions.

- **`grappa_loss` block**:
  - **`node_loss` block**:
    - **`name: shower_primary`**: Specifies that the primary loss is for shower segmentation.
  - **`edge_loss` block**:
    - **`name: channel`**: Specifies the edge loss based on the "channel" connection between nodes.

### Purpose
The `grappa_shower.cfg` file sets up the GRAPPA model for 3D cluster-based training, primarily focusing on recognizing shapes like showers, Michel electrons, and delta rays. The file controls the training process, specifying how the dataset is loaded, how the model is configured, and how the training is performed. It trains the model for 25,000 iterations and saves checkpoints every 1,000 iterations while utilizing various geometric features for both node and edge encoding.


## Overview of `train_uresnet.sh` Script

The `train_uresnet.sh` script is a SLURM batch job submission script used to train the **UResNet** model on a GPU cluster. It specifies the job configuration and execution details for running the training process using **Singularity** and **SPINE**.

### SLURM Configuration

The first part of the script defines the SLURM job options:

- **`--account=neutrino:ml-dev`**: Specifies the account to charge the GPU usage to, in this case, the neutrino machine learning development account.
- **`--partition=ampere`**: Specifies the partition to run the job in. The "ampere" partition corresponds to nodes with **Ampere GPUs**.
- **`--job-name=train_uresnet`**: Sets the job name as "train_uresnet," which will be visible in SLURM job listings.
- **`--output=batch_outputs/output-train_uresnet.txt`**: Specifies the output file for stdout messages during the job execution.
- **`--error=batch_outputs/output-train_uresnet.txt`**: Specifies the output file for stderr messages during the job execution.
  
- **`--ntasks=1`**: Specifies the number of tasks to run (1 task).
- **`--cpus-per-task=5`**: Allocates 5 CPU cores for the task.
- **`--mem-per-cpu=4g`**: Allocates 4 GB of RAM per CPU core (total of 20 GB for the job).
- **`--time=6:00:00`**: Sets the maximum runtime for the job to 6 hours.
- **`--gpus=a100:1`**: Requests 1 A100 GPU for the job.

### Singularity and Execution Command

The core of the script executes the training using **Singularity**:

- **`singularity exec`**: Executes a command within a Singularity container.
  - **`--bind /sdf/group/neutrino/drielsma/,/sdf/data/neutrino/`**: Binds the directories to the container, ensuring that the necessary files for the training are accessible from the Singularity environment.
  - **`--nv`**: Enables NVIDIA GPU support inside the container.
  - **`/sdf/group/neutrino/images/develop.sif`**: Specifies the Singularity image (`develop.sif`) that contains the necessary software and dependencies for training.
  
- **`bash -c`**: Executes a command within a bash shell.
  - **`"python3 /sdf/data/neutrino/software/spine/bin/run.py -c /sdf/group/neutrino/drielsma/train/example/uresnet.cfg"`**: 
    - **`python3`**: Runs the Python interpreter.
    - **`/sdf/data/neutrino/software/spine/bin/run.py`**: Specifies the SPINE executable script to run the UResNet training.
    - **`-c /sdf/group/neutrino/drielsma/train/example/uresnet.cfg`**: Specifies the configuration file (`uresnet.cfg`) that contains the details for the UResNet model training.

### Purpose

This script automates the submission of a training job for the **UResNet** model to a GPU cluster. It leverages **Singularity** to encapsulate the required environment, and **SLURM** to manage the job scheduling, resource allocation, and execution. The training is configured to use an A100 GPU and is expected to run for a maximum of 6 hours, saving both standard output and error messages to the specified log files.


## Overview of `val_uresnet.sh` Script

The `val_uresnet.sh` script is a SLURM batch job submission script used to validate the **UResNet** model on a GPU cluster. This script specifies the necessary configuration and execution details for running the validation process using **Singularity** and **SPINE**.

### SLURM Configuration

The first section of the script contains SLURM job options, which define how the job should be executed on the cluster:

- **`--account=neutrino:ml-dev`**: The account under which the GPU usage is charged. Here, it's for the neutrino machine learning development group.
- **`--partition=ampere`**: Specifies the partition for the job. The "ampere" partition is for nodes with **Ampere GPUs**.
- **`--job-name=val_uresnet`**: The job name is set as "val_uresnet," which helps identify this validation job in SLURM job queues.
- **`--output=batch_outputs/output-val_uresnet.txt`**: Defines where the output log (stdout) will be saved.
- **`--error=batch_outputs/output-val_uresnet.txt`**: Defines where the error log (stderr) will be saved.
  
- **`--ntasks=1`**: Specifies that only 1 task will be executed.
- **`--cpus-per-task=5`**: Allocates 5 CPU cores for this task (which helps in loading and processing data).
- **`--mem-per-cpu=4g`**: Allocates 4 GB of RAM per CPU core, giving a total of 20 GB for the job.
- **`--time=0:15:00`**: Sets the maximum runtime to 15 minutes, which is sufficient for the validation process.
- **`--gpus=a100:1`**: Requests 1 A100 GPU for the job, which is required to run the model.

### Singularity and Execution Command

The core of the script runs the validation using **Singularity**:

- **`singularity exec`**: Executes the validation process inside a Singularity container.
  - **`--bind /sdf/group/neutrino/drielsma/,/sdf/data/neutrino/`**: Binds the necessary directories, allowing access to the data and configuration files inside the Singularity container.
  - **`--nv`**: Enables the use of Nvidia GPUs within the container.
  - **`/sdf/group/neutrino/images/develop.sif`**: Points to the Singularity image (`develop.sif`), which contains the required software and dependencies for running the UResNet validation.

- **`bash -c`**: Executes the command in a bash shell:
  - **`python3 /sdf/data/neutrino/software/spine/bin/run.py -c /sdf/group/neutrino/drielsma/train/example/uresnet_val.cfg`**: 
    - **`python3`**: Runs the Python interpreter.
    - **`/sdf/data/neutrino/software/spine/bin/run.py`**: Specifies the SPINE executable script to run the UResNet validation.
    - **`-c /sdf/group/neutrino/drielsma/train/example/uresnet_val.cfg`**: Specifies the validation configuration file (`uresnet_val.cfg`), which contains the details for running the UResNet validation process.

### Purpose

This script automates the submission of a validation job for the **UResNet** model on a GPU cluster. The job runs for 15 minutes, validating the model using the pre-trained weights, and saving both standard output and error logs to specific files. The validation process is run inside a Singularity container, ensuring that all dependencies are properly managed, while **SLURM** handles the scheduling and resource allocation.


# Workflow Explanation of Training and Validation Configuration Files

This document explains the workflow of the configuration files (`uresnet.cfg`, `uresnet_val.cfg`, `grappa_shower.cfg`, `train_uresnet.sh`, and `val_uresnet.sh`) in conjunction with the **Training_validation.ipynb** notebook.

## 1. Training_validation.ipynb

This notebook serves as a detailed guide to understanding the various configuration files used for training and validating machine learning models in the **SPINE** framework. It walks through the basic concepts of the configuration files and how to execute the training and validation processes on a GPU cluster. The notebook focuses on **UResNet**, a deep neural network architecture used for semantic segmentation.

---

## 2. uresnet.cfg

### Purpose:
This file is used to define the configuration for training the **UResNet** model on sparse 3D data, likely for a neutrino detection task. It specifies how the data is loaded, how the model is structured, and how training is performed.

### Workflow:
1. **Base Configuration**:
   - The training iterations, logging directory, and the optimizer (Adam with a learning rate of 0.001) are specified.
   - The UResNet model is set to train for 25,000 iterations, saving checkpoints every 1,000 iterations.

2. **IO Configuration**:
   - The data is loaded from a sparse 3D dataset located at `/sdf/data/neutrino/generic/mpvmpr_2020_01_v04/train.root`.
   - The data loader prepares the batch size (128) and uses 4 CPU cores to load data efficiently.

3. **Model Configuration**:
   - The model is configured as **UResNet**, with the architecture specified in terms of layers, filters, and activation functions.
   - No pre-trained weights are used, and the model is trained from scratch.

### Execution:
- This configuration file is loaded in the **Training_validation.ipynb** notebook to guide the training process. In the notebook, this configuration is discussed and broken down into smaller pieces for better understanding (Cells 4 to 6).
- The training is triggered using `train_uresnet.sh`, which submits a job to the SLURM scheduler for GPU-based training.

---

## 3. uresnet_val.cfg

### Purpose:
This file is used for validating the UResNet model using previously saved weights. It is similar to the `uresnet.cfg` but with important modifications for the validation process.

### Workflow:
1. **Base Configuration**:
   - The number of iterations is set to 100 (since the model is being validated, not trained), and the log directory remains the same as for training.

2. **IO Configuration**:
   - The data is now loaded from a **test set** at `/sdf/data/neutrino/generic/mpvmpr_2020_01_v04/test.root` to ensure that the model is being validated on unseen data.

3. **Model Configuration**:
   - The model architecture is the same as in the training configuration, but now the path to the pre-trained model weights (`weights/uresnet/snapshot*.ckpt`) is specified for the validation process.

### Execution:
- After the training process is complete, this configuration file is used to validate the model. The notebook guides the process of setting up this configuration (Cell 28), and the validation job is submitted via the `val_uresnet.sh` script, which runs the validation using **Singularity** on a GPU.
- The validation logs are automatically stored in the log directory specified (`logs/uresnet`).

---

## 4. grappa_shower.cfg

### Purpose:
This configuration file is used to train another model, **Grappa**, which is likely focused on particle clustering or shower detection tasks, with data structured differently than UResNet.

### Workflow:
1. **Base Configuration**:
   - The training is set for 25,000 iterations, saving weights every 1,000 steps, similar to the UResNet configuration. 

2. **IO Configuration**:
   - The dataset is specified as a 3D cluster dataset (`cluster3d`), with additional particle information such as corrected particle events and clean data.

3. **Model Configuration**:
   - The **Grappa** model is defined with specific node and edge encoders and a GNN-based architecture to process the input data.
   - The configuration is tailored for handling shower, Michel, and delta particle shapes.

### Execution:
- This configuration can be run in a similar manner to the UResNet training, by modifying the script or using a new SLURM script to handle the training process for this model.
- Like UResNet, it uses the same general workflow: data loading, model training, and logging.

---

## 5. train_uresnet.sh

### Purpose:
This SLURM job submission script automates the process of training the UResNet model on the cluster.

### Workflow:
1. The script sets up the required job resources, including:
   - Allocation of 5 CPU cores and 1 A100 GPU for the job.
   - Setting the job name (`train_uresnet`) and output/error log paths.
   - Running the job for a maximum of 6 hours.

2. The script launches the UResNet training by running the **SPINE** framework inside a **Singularity** container, binding the required directories and executing the training using the `uresnet.cfg` configuration file.

### Execution:
- The script is submitted using the `sbatch train_uresnet.sh` command, which schedules the training job on the cluster.
- The output and error logs can be monitored to check the progress of the training process.

---

## 6. val_uresnet.sh

### Purpose:
This script is similar to `train_uresnet.sh`, but it is used for validating the UResNet model on the test dataset.

### Workflow:
1. The SLURM job is set up similarly, with a shorter time limit of 15 minutes, as validation typically requires fewer resources and time compared to training.

2. The script runs the validation using the `uresnet_val.cfg` configuration file inside a **Singularity** container, loading the saved model weights for validation on the test set.

### Execution:
- This script is executed with the `sbatch val_uresnet.sh` command, running the validation process.
- The validation logs are saved, and the validation results (accuracy, loss, etc.) are analyzed.

---

## Overall Workflow Integration

### 1. Training (UResNet or Grappa):
- The training configuration files (`uresnet.cfg` or `grappa_shower.cfg`) are loaded, specifying how data is handled and how the model is trained.
- The training process is initiated by submitting the `train_uresnet.sh` script, which schedules the job on the cluster.
- Training logs are generated and saved to monitor the model’s progress.

### 2. Validation:
- Once the training is complete, the validation configuration (`uresnet_val.cfg`) is used to test the model's performance on the test dataset.
- The `val_uresnet.sh` script submits the validation job to SLURM, and the validation logs are saved.

### 3. Analysis (via Notebook):
- The **Training_validation.ipynb** notebook is used to visualize the training and validation logs, analyze the loss and accuracy, and ensure that the model is learning properly.
- The notebook also provides step-by-step instructions for adjusting configurations and submitting jobs.

---

This workflow allows for a streamlined process of training, validating, and analyzing deep learning models like UResNet in a high-performance computing environment, utilizing SLURM, Singularity, and SPINE.


# Workflow Adjustments for Argonne Polaris

Since you're working on Argonne Polaris and can download files from SDF, here’s a list of files you need to download and the changes you need to make to point to your new data location on Polaris.

## Files to Download from SDF

1. **Training Data**
   - From: `/sdf/data/neutrino/generic/mpvmpr_2020_01_v04/train.root`
   - To: Polaris local directory

2. **Validation Data**
   - From: `/sdf/data/neutrino/generic/mpvmpr_2020_01_v04/test.root`
   - To: Polaris local directory

3. **Configuration Files**
   - Download these configuration and shell script files:
     - UResNet: `uresnet.cfg` and `uresnet_val.cfg`
     - GRAPPA: `grappa_shower.cfg`
     - Shell scripts: `train_uresnet.sh`, `val_uresnet.sh`

## Changes to Make for Polaris

In the configuration files (`uresnet.cfg`, `uresnet_val.cfg`, `grappa_shower.cfg`), modify the paths to reflect the new location of your data on Polaris.

### 1. uresnet.cfg
Original:
```yaml
file_keys: /sdf/data/neutrino/generic/mpvmpr_2020_01_v04/train.root
```
Change to:
```yaml
file_keys: /your/polaris/local/path/train.root
```

### 2. uresnet.cfg
Original:
```yaml
file_keys: /sdf/data/neutrino/generic/mpvmpr_2020_01_v04/test.root
```
Change to:
```yaml
file_keys: /your/polaris/local/path/test.root
```

### 3. grappa_shower.cfg
Original:
```yaml
file_keys: /sdf/data/neutrino/generic/mpvmpr_2020_01_v04/train.root
```
Change to:
```yaml
file_keys: /your/polaris/local/path/train.root
```